<a href="https://colab.research.google.com/github/dpierso1/Pierson_IT533/blob/main/Week_9_Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import spatial
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)



In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import spatial
import statsmodels.api as sm


In [ ]:
# We import all the kNN libraries

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

from sklearn import neighbors, datasets
from sklearn.model_selection import cross_val_score, train_test_split
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore") 
np.random.seed(42)
# Data
airline = pd.read_csv("https://raw.githubusercontent.com/dpierso1/Pierson_IT533/main/airline_passenger_satisfaction.csv", encoding="ISO-8859-1") # USE YOUR OWN DATA ON GITHUB
airline.dropna(inplace=True)
airline.head(5)

,ID,Gender,Age,CustomerType,TypeOfTravel,Class,FlightDistance,DepartureDelay,ArrivalDelay,EaseOfOnlineBooking,...,On-boardService,SeatComfort,LegRoom,Cleanliness,FoodAndDrink,In-flightWifi,In-flightEntertainment,BaggageHandling,PriceUSD,Satisfaction
0,1,Male,48,First-time,Business,Business,821,2,5.0,3,...,3,5,2,5,5,3,5,5,1053.59,Neutral or Dissatisfied
1,2,Female,35,Returning,Business,Business,821,26,39.0,2,...,5,4,5,5,3,2,5,5,1051.94,Satisfied
2,3,Male,41,Returning,Business,Business,853,0,0.0,4,...,3,5,3,5,5,4,3,3,1627.35,Satisfied
3,4,Male,50,Returning,Business,Business,1905,0,0.0,2,...,5,5,5,4,4,2,5,5,2520.45,Satisfied
4,5,Female,49,Returning,Business,Business,3470,0,1.0,3,...,3,4,4,5,4,3,3,3,6048.50,Satisfied


In [ ]:
airline_nn = pd.DataFrame(airline, columns = ["Age", 'ID', 'FlightDistance', 'PriceUSD', 'Class'])
airline_nn.head()

,Age,ID,FlightDistance,PriceUSD,Class
0,48,1,821,1053.59,Business
1,35,2,821,1051.94,Business
2,41,3,853,1627.35,Business
3,50,4,1905,2520.45,Business
4,49,5,3470,6048.50,Business


In [ ]:
# Replace Business with 0, First Class with 1, Economy with 2
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
airline_nn["Class"] = labelencoder.fit_transform(airline_nn["Class"])
region = pd.DataFrame({'Class': ['Business', 'Economy', 'First Class']})
airline_nn.head()

,Age,ID,FlightDistance,PriceUSD,Class
0,48,1,821,1053.59,0
1,35,2,821,1051.94,0
2,41,3,853,1627.35,0
3,50,4,1905,2520.45,0
4,49,5,3470,6048.50,0


In [ ]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df
airline_nn = create_dummies(airline_nn,"Class")

airline_nn.head()

,Age,ID,FlightDistance,PriceUSD,Class,Class_0,Class_1,Class_2
0,48,1,821,1053.59,0,1,0,0
1,35,2,821,1051.94,0,1,0,0
2,41,3,853,1627.35,0,1,0,0
3,50,4,1905,2520.45,0,1,0,0
4,49,5,3470,6048.50,0,1,0,0


In [ ]:
# Features before mean normalization
unscaled_features = airline_nn[['Age','ID','FlightDistance','PriceUSD']]

# Mean Normalization to have a faster classifier 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# Calculate μ & σ(fit) and apply the transformation(transform)
unscaled_features_array = sc.fit_transform(unscaled_features.values)

# Assign the scaled data to a DataFrame & use the index and columns arguments to keep your original indices and column names:
scaled_features = pd.DataFrame(unscaled_features_array, index=unscaled_features.index, columns=unscaled_features.columns)

scaled_features.head()

,Age,ID,FlightDistance,PriceUSD
0,0.567460,-1.732671,-0.361689,-0.396247
1,-0.292821,-1.732643,-0.361689,-0.397299
2,0.104232,-1.732616,-0.329591,-0.030385
3,0.699811,-1.732588,0.725643,0.539106
4,0.633636,-1.732561,2.295454,2.788790


In [ ]:
from sklearn.model_selection import train_test_split
X = scaled_features
y = airline_nn['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

125743

In [ ]:
from sklearn.neural_network import MLPClassifier

# Initializing the multilayer perceptron
# running this with all the default settings, including a hidden_layer_sizes setting of 100 adn 200 maximum iterations. 
# Verbose is on so we can see how this runs.

mlp1 = MLPClassifier(verbose=1) # We are running this in verbose mode so we can see the output of each iteration.

In [ ]:
# As this runs in verbose mode, note how the loss function steps down with every single iteration!

mlp1.fit(X_train, y_train)

Iteration 1, loss = 0.74250512
Iteration 2, loss = 0.69913148
Iteration 3, loss = 0.68889314
Iteration 4, loss = 0.68235798
Iteration 5, loss = 0.67788321
Iteration 6, loss = 0.67443089
Iteration 7, loss = 0.67153772
Iteration 8, loss = 0.66901549
Iteration 9, loss = 0.66620450
Iteration 10, loss = 0.66385249
Iteration 11, loss = 0.66115406
Iteration 12, loss = 0.65825854
Iteration 13, loss = 0.65522147
Iteration 14, loss = 0.65170126
Iteration 15, loss = 0.64858394
Iteration 16, loss = 0.64594299
Iteration 17, loss = 0.64313008
Iteration 18, loss = 0.64029913
Iteration 19, loss = 0.63756095
Iteration 20, loss = 0.63522194
Iteration 21, loss = 0.63306333
Iteration 22, loss = 0.63096724
Iteration 23, loss = 0.62879640
Iteration 24, loss = 0.62729298
Iteration 25, loss = 0.62530938
Iteration 26, loss = 0.62394733
Iteration 27, loss = 0.62225525
Iteration 28, loss = 0.62097506
Iteration 29, loss = 0.61946904
Iteration 30, loss = 0.61806303
Iteration 31, loss = 0.61733224
Iteration 32, los

MLPClassifier(verbose=1)

In [ ]:
# Score takes a feature matrix X_test and the expected target values y_test. 
# Predictions for X_test are compared with y_test

print (mlp1.score(X_test,y_test))

0.7356612685560054


In [ ]:
!pip install matplotlib-venn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


In [ ]:
# Now, let's see what the predictions look like
y_pred = mlp1.predict(X_test)
print("Test set predictions: \n {}".format(y_pred))

Test set predictions: 
 [1 1 0 ... 0 1 1]


In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.78      0.78     20307
           1       0.69      0.80      0.74     19307
           2       0.00      0.00      0.00      3139

    accuracy                           0.73     42753
   macro avg       0.49      0.53      0.51     42753
weighted avg       0.68      0.73      0.71     42753



In [ ]:
mlp1 = MLPClassifier(hidden_layer_sizes=10,solver='sgd',learning_rate_init= 0.01, max_iter=500, verbose = 1)

In [ ]:
airline_nn.shape[0]

125743